In [1]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset
import torch


dataset_name = 'ogbn-arxiv'

dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
print('The {} dataset has {} graph'.format(dataset_name, len(dataset)))

# Extract the graph
data = dataset[0]
print(data)

# 查看节点ID范围
print(f"\n节点ID范围: 0 ~ {data.x.shape[0]-1}")
# 查看标签信息
print("\n标签信息:")
print(f"标签维度: {data.y.shape}")
print(f"标签取值范围: {torch.min(data.y).item()} ~ {torch.max(data.y).item()}")
print(f"标签类别数量: {len(torch.unique(data.y))}")


The ogbn-arxiv dataset has 1 graph
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])

节点ID范围: 0 ~ 169342

标签信息:
标签维度: torch.Size([169343, 1])
标签取值范围: 0 ~ 39
标签类别数量: 40


In [5]:
from torch_geometric.utils import degree
from torch_sparse import SparseTensor

# 获取边索引
edge_index = torch.stack([data.adj_t.storage.row(), data.adj_t.storage.col()], dim=0)

# 计算总边数（直接使用edge_index的列数）
total_edges = edge_index.size(1)

# 计算自环数量
self_loops = (edge_index[0] == edge_index[1]).sum().item()

# 计算重复边
edge_list = list(zip(edge_index[0].tolist(), edge_index[1].tolist()))
unique_edges = len(set(edge_list))
duplicate_edges = total_edges - unique_edges

# 计算度数（保持原有的度数统计）
degrees = degree(edge_index[1], num_nodes=data.num_nodes)

print("\n边的统计信息:")
print(f"总边数: {total_edges}")
print(f"自环数量: {self_loops}")
print(f"重复边数量: {duplicate_edges}")
print(f"唯一边数量: {unique_edges}")


# 统计度数信息
print("\n度数统计信息:")
print(f"平均度数: {degrees.mean().item():.2f}")
print(f"最大度数: {degrees.max().item()}")
print(f"最小度数: {degrees.min().item()}")
print(f"度数标准差: {degrees.std().item():.2f}")

# 找出度数为0的节点(孤立节点)
isolated_nodes = torch.where(degrees == 0)[0]

print(f"\n孤立节点数量: {len(isolated_nodes)}")
if len(isolated_nodes) > 0:
    print("存在孤立节点,其ID为:")
    print(isolated_nodes)
else:
    print("不存在孤立节点")



边的统计信息:
总边数: 1166243
自环数量: 0
重复边数量: 0
唯一边数量: 1166243

度数统计信息:
平均度数: 6.89
最大度数: 436.0
最小度数: 0.0
度数标准差: 8.88

孤立节点数量: 17440
存在孤立节点,其ID为:
tensor([     3,      5,      6,  ..., 169098, 169099, 169251])
